# Creating Fresh Data Base

In [7]:
import psycopg2

def get_create_table_queries(conn):
    create_table_queries = []
    try:
        # Create a cursor object
        cur = conn.cursor()

        # Get the list of all schemas excluding temporary and system schemas
        schema_sql = """
        SELECT schema_name 
        FROM information_schema.schemata
        WHERE schema_name NOT LIKE 'pg_temp_%'
        AND schema_name NOT LIKE 'pg_toast%'
        AND schema_name NOT LIKE 'information%'
        AND schema_name NOT LIKE 'pg_catalog%'
        """
        cur.execute(schema_sql)
        schemas = cur.fetchall()

        # Get the CREATE TABLE statements for all tables in each schema
        for schema in schemas:
            schema_name = schema[0]
            table_sql = f"""
            SELECT table_name 
            FROM information_schema.tables
            WHERE table_schema = '{schema_name}'
            """
            cur.execute(table_sql)
            tables = cur.fetchall()

            for table in tables:
                table_name = table[0]
                create_sql = f"""
                SELECT 'CREATE TABLE ' || quote_ident('{schema_name}') || '.' || quote_ident('{table_name}') || E'\n(\n' ||
                array_to_string(
                    array_agg(
                        '    ' || quote_ident(column_name) || ' ' || data_type ||
                        coalesce('(' || character_maximum_length || ')', '') ||
                        case when is_nullable = 'NO' then ' NOT NULL' else '' end
                    ), E',\n'
                ) || E'\n);\n'
                FROM information_schema.columns
                WHERE table_schema = '{schema_name}' AND table_name = '{table_name}'
                GROUP BY table_schema, table_name
                """
                cur.execute(create_sql)
                create_table_query = cur.fetchone()[0]
                create_table_queries.append(create_table_query)

        # Close the cursor
        cur.close()

    except Exception as e:
        print(f"Error: {e}")

    return create_table_queries

def save_to_sql_file(filename, queries):
    with open(filename, 'w') as f:
        for query in queries:
            f.write(query + '\n\n')


conn = psycopg2.connect(
    host="192.168.2.54",
    port=5432,
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

# create the schemas


create_table_queries = get_create_table_queries(conn)
save_to_sql_file("create_tables.sql", create_table_queries)


# Close the connection
conn.close()

# Run this cell to create all the tables and schemas

In [2]:
import psycopg2
# connect to local host db named BravisaDB
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="BravisaDB",
    user="postgres",
    password="edsols"
)

# run the create_table.sql file 
with open("create_tables.sql", 'r') as f:
    create_table_sql = f.read()
    cur = conn.cursor()
    cur.execute(create_table_sql)
    conn.commit()
    cur.close()
    

In [17]:
import os

cwd = os.getcwd()

ohlc = os.path.join(cwd, "app\\OHLCFiles")

# for files starting with cm and ending with .csv rename then to upper()
for file in os.listdir(ohlc):
    if file.startswith("cm") and file.endswith(".csv"):
        os.rename(os.path.join(ohlc, file), os.path.join(ohlc, file.upper()))

In [17]:
import datetime
import os
import pandas as pd

cwd = os.getcwd()

ohlc = os.path.join(cwd, "app\\OHLCFiles\\")
indexfiles = os.path.join(cwd, "app\\IndexOHLCFiles\\")

# pandas date_range
date_range = pd.date_range(start='01-01-2020', end='18-10-2024', freq='D')

for curr_date in date_range:
    # Format the date for NSE file naming convention
    download_date_nse = curr_date.strftime("%d%b%Y").upper()
    file_nse = f'CM{download_date_nse}BHAV.CSV'

    # Format the date for EQ_ISINCODE file naming convention
    download_date_bse = curr_date.strftime("%d%m%y")
    file_bse = f"EQ_ISINCODE_{download_date_bse}.CSV"

    ohlc_files = os.listdir(ohlc)


    #if file_nse and file_bse are both not in the ohlc_files list
    # if file_nse not in ohlc_files and file_bse not in ohlc_files:
    #     # ignore saturday and sunday
    #     if(curr_date.weekday() == 5 or curr_date.weekday() == 6):
    #         continue
    #     print(f"Both {file_nse} and {file_bse} are not present in the directory")
    # else:
    #     if file_nse not in ohlc_files:
    #         print(f"{file_nse} is not present in the directory")

    #     if file_bse not in ohlc_files:
    #         print(f"{file_bse} is not present in the directory")

    # ind_close_all_01012020.csv check for this file in the IndexOHLCFiles directory
    download_date_index = curr_date.strftime("%d%m%Y")
    file_index = f"ind_close_all_{download_date_index}.csv"

    index_files = os.listdir(indexfiles)

    if file_index not in index_files:
        if(curr_date.weekday() == 5 or curr_date.weekday() == 6):
            continue
        print(f"{file_index} is not present in the directory")
    



ind_close_all_21022020.csv is not present in the directory
ind_close_all_10032020.csv is not present in the directory
ind_close_all_02042020.csv is not present in the directory
ind_close_all_06042020.csv is not present in the directory
ind_close_all_10042020.csv is not present in the directory
ind_close_all_14042020.csv is not present in the directory
ind_close_all_01052020.csv is not present in the directory
ind_close_all_25052020.csv is not present in the directory
ind_close_all_02102020.csv is not present in the directory
ind_close_all_16112020.csv is not present in the directory
ind_close_all_30112020.csv is not present in the directory
ind_close_all_25122020.csv is not present in the directory
ind_close_all_26012021.csv is not present in the directory
ind_close_all_11032021.csv is not present in the directory
ind_close_all_29032021.csv is not present in the directory
ind_close_all_02042021.csv is not present in the directory
ind_close_all_14042021.csv is not present in the directo

In [39]:
import psycopg2
from collections import Counter
# Expected data types for each column by table name
expected_data_types = {
    "FRS-NAVCategoryAvg": {
    "btt_scheme_category": str,
    "Date": pd.to_datetime,
    "1 Day Average": float,
    "1 Week Average": float,
    "1 Month Average": float,
    "3 Month Average": float,
    "6 Month Average": float,
    "9 Month Average": float,
    "1 Year Average": float,
    "2 Year Average": float,
    "3 Year Average": float,
    "5 Year Average": float,
},
"FRS-MFRank":    {
    "CompanyCode": float,
    "CompanyName": str,
    "Date": pd.to_datetime,
    "ISINCode": str,
    "Quantity": float,
    "OutstandingShares": float,
    "MFExposure": float,
    "MFRank": float,
},
"CombinedRS" : {
    "CompanyCode": float,
    "CompanyName": str,
    "NSECode": str,
    "BSECode": int,
    "PRSRank": float,
    "ERSRank": float,
    "RRSRank": float,
    "FRSRank": float,
    "IRSRank": float,
    "CombiRS": float,
    "Rank": int,
    "Value Average": float,
    "GenDate": pd.to_datetime,
},
"AnnualGeneralMeeting": {
    "CompanyCode": float,
    "DateOfAnnouncement": pd.to_datetime,
    "AGMDate": pd.to_datetime,
    "YearEnding": pd.to_datetime,
    "Purpose": str,
    "RecordDate": pd.to_datetime,
    "BookClosureStartDate": pd.to_datetime,
    "BookClosureEndDate": pd.to_datetime,
    "Remarks": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"Auditors": {
    "Companycode": float,
    "AgencyName": str,
    "ModifiedDate": pd.to_datetime,
},
"BSE": {
    "SC_CODE": float,
    "SC_NAME": str,
    "SC_GROUP": str,
    "SC_TYPE": str,
    "OPEN": float,
    "HIGH": float,
    "LOW": float,
    "CLOSE": float,
    "LAST": float,
    "PREVCLOSE": float,
    "NO_TRADES": float,
    "NO_OF_SHRS": float,
    "NET_TURNOV": float,
    "TDCLOINDI": str,
    "ISIN_CODE": str,
    "TRADING_DATE": pd.to_datetime,
},
"BTTDivisor": {
    "IndexName": str,
    "BTTIndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": int,
},
"BTTDivisor_DUPLICATED": {
    "IndexName": str,
    "BTTIndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": int,
},
"BTTList": {
    "CompanyName": str,
    "ISIN": str,
    "NSECode": str,
    "BSECode": float,
    "Date": pd.to_datetime,
    "MFList": bool,
    "BTTDate": pd.to_datetime,
    "CompanyCode": float,
},
"BTTListCopy": {
    "CompanyName": str,
    "ISIN": str,
    "NSECode": str,
    "BSECode": int,
    "Date": pd.to_datetime,
    "MFList": bool,
    "BTTDate": pd.to_datetime,
    "CompanyCode": float,
},
"BTTList_DUPLICATED": {
    "CompanyName": str,
    "ISIN": str,
    "NSECode": str,
    "BSECode": int,
    "Date": pd.to_datetime,
    "MFList": bool,
    "BTTDate": pd.to_datetime,
    "CompanyCode": float,
    "Updated": str,
},
"BTTList_STABLE": {
    "CompanyName": str,
    "ISIN": str,
    "NSECode": str,
    "BSECode": float,
    "Date": pd.to_datetime,
    "MFList": bool,
    "BTTDate": pd.to_datetime,
    "CompanyCode": float,
},
"BackgroundInfo": {
    "CompanyCode": float,
    "CompanyName": str,
    "ShortCompanyName": str,
    "TickerName": str,
    "ParentCompanyCode": float,
    "PersonDescription": str,
    "PersonName": str,
    "ExecutiveDescription": str,
    "ExecutiveName": str,
    "BSECode": int,
    "BSEGroup": str,
    "NSECode": str,
    "ISINCode": str,
    "Sourcetype": str,
    "IncorporationDate": pd.to_datetime,
    "FirstPublicIssueDate": pd.to_datetime,
    "Auditeddate": pd.to_datetime,
    "FaceValue": float,
    "Marketlot": float,
    "IndustryCode": int,
    "IndustryName": str,
    "BusinessGroupCode": int,
    "BusinessGroupName": str,
    "MajorSector": int,
    "latestFinYear": str,
    "ForemostFinYear": str,
    "LatestHalfYear": str,
    "ForemostHalfYear": str,
    "LatestQtrYear": str,
    "ForemostQtrYear": str,
    "ModifiedDate": pd.to_datetime,
},
"Bankers": {
    "Companycode": float,
    "AgencyName": str,
    "ModifiedDate": pd.to_datetime,
},
"Blockdeals": {
    "CompanyCode": float,
    "Exchange": str,
    "Symbol": str,
    "DealDate": pd.to_datetime,
    "DealType": str,
    "ClientName": str,
    "Quantity": int,
    "Price": float,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"BoardMeetings": {
    "CompanyCode": float,
    "BoardMeetDate": pd.to_datetime,
    "Purpose": str,
    "Remarks": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"Bonus": {
    "CompanyCode": float,
    "DateOfAnnouncement": pd.to_datetime,
    "RatioExisting": float,
    "RatioOfferred": float,
    "RecordDate": pd.to_datetime,
    "BookClosureStartDate": pd.to_datetime,
    "BookClosureEndDate": pd.to_datetime,
    "XBDate": pd.to_datetime,
    "Remarks": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"BulkDeals": {
    "CompanyCode": float,
    "Exchange": str,
    "Symbol": str,
    "DealDate": pd.to_datetime,
    "DealType": str,
    "ClientName": str,
    "Quantity": int,
    "Price": float,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"CapitalStructure": {
    "companycode": float,
    "FromYear": float,
    "ToYear": float,
    "InstrumentName": str,
    "ReferencePercentage": float,
    "AuthorizedShares": float,
    "AuthorizedFaceValue": float,
    "AuthorizedCapital": float,
    "IssuedShares": float,
    "IssuedCapital": float,
    "NoOfPaidUpShares": float,
    "PaidUpFaceValue": float,
    "PaidUpCapital": float,
    "TotalArrears": float,
    "Reasonsforchange": str,
    "ModifiedDate": pd.to_datetime,
},
"CompanyMaster": {
    "CompanyCode": float,
    "CompanyName": str,
    "ShortCompanyName": str,
    "IndustryCode": int,
    "IndustryName": str,
    "BusinessGroupCode": int,
    "BusinessGroupName": str,
    "IncorporationDate": pd.to_datetime,
    "IncorporationYear": str,
    "FirstPublicIssueDate": pd.to_datetime,
    "FirstPublicIssueYear": str,
    "MajorSector": int,
    "CINNo": str,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag ": bool,
},
"CompanyNameChange": {
    "CompanyCode": float,
    "EffectiveDate": pd.to_datetime,
    "OldCompanyName": str,
    "NewCompanyName": str,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedHalfYearlyResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Half": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "MinorityInterest": float,
    "ShareOfPLOfAssociates": float,
    "NetPLAfterMIAssociates": float,
    "CostOfInvestInSubsidiary": float,
    "PriorYearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Per_OfGrossNPA": float,
    "Per_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "asBeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedNineMonthsResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Nine": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "MinorityInterest": float,
    "ShareOfPLOfAssociates": float,
    "NetPLAfterMIAssociates": float,
    "CostOfInvestInSubsidiary": float,
    "PriorYearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Per_OfGrossNPA": float,
    "Per_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "asBeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedQuarterlyEPS": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PATRAW": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
    "Ext_Flag": bool,
    "Q1 EPS Growth": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
},
"ConsolidatedQuarterlyEPS_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PATRAW": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
    "Ext_Flag": bool,
    "Q1 EPS Growth": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
},
"ConsolidatedQuarterlyResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "MinorityInterest": float,
    "ShareOfPLOfAssociates": float,
    "NetPLAfterMIAssociates": float,
    "CostOfInvestInSubsidiary": float,
    "PriorYearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Per_OfGrossNPA": float,
    "Per_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "asBeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedQuarterlyResults_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "MinorityInterest": float,
    "ShareOfPLOfAssociates": float,
    "NetPLAfterMIAssociates": float,
    "CostOfInvestInSubsidiary": float,
    "PriorYearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Per_OfGrossNPA": float,
    "Per_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "asBeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedQuarterlyResults_OLD": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "MinorityInterest": float,
    "ShareOfPLOfAssociates": float,
    "NetPLAfterMIAssociates": float,
    "CostOfInvestInSubsidiary": float,
    "PriorYearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Per_OfGrossNPA": float,
    "Per_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "asBeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"ConsolidatedTTM": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"ConsolidatedTTM_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"CurrentData": {
    "Companycode": float,
    "ResultType": str,
    "Yearending": pd.to_datetime,
    "Months": int,
    "Facevalue": float,
    "BonusEquity": float,
    "NumberOfEquityShares": float,
    "CurrentEquity": float,
    "CurrentReserves": float,
    "TotalDebt": float,
    "CashAndBankBalance": float,
    "Sales": float,
    "PrevYearSales": float,
    "OperatingProfit": float,
    "GrossProfit": float,
    "Depriciation": float,
    "Tax": float,
    "ReportedPAT": float,
    "PrevYearReportedPAT": float,
    "Dividend": float,
    "DividendPerShare": float,
    "EPS": float,
    "BookValue": float,
    "Price": float,
    "PriceDate": pd.to_datetime,
    "IndustryPE": float,
    "BetaValue": float,
    "PledgedShares": float,
    "TrailingYear": pd.to_datetime,
    "LastFourQuarterersSales": float,
    "LastFourQuartersTotalExpenditure": float,
    "LastFourQuartersInterestCharges": float,
    "LastFourQuartersGrossProfit": float,
    "LastFourQuartersDepreciation": float,
    "LastFourQuartersBankProvisionsMade": float,
    "LastFourQuartersTaxCharges": float,
    "LastFourQuartersExtraOrdinaryItem": float,
    "LastFourQuartersReportedPAT": float,
    "LastFourQuartersPriorYearAdj": float,
    "TrailingEPS": float,
    "TrailingCEPS": float,
    "ModifiedDate": pd.to_datetime,
},
"Dividend": {
    "CompanyCode": float,
    "DateOfAnnouncement": pd.to_datetime,
    "InterimOrFinal": str,
    "InstrumentType": int,
    "InstrumentTypeDescription": str,
    "Percentage": float,
    "Value": float,
    "RecordDate": pd.to_datetime,
    "BookClosureStartDate": pd.to_datetime,
    "BookClosureEndDate": pd.to_datetime,
    "XDDate": pd.to_datetime,
    "Remarks": str,
    "DelFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"ErrorLog": {
    "TIMESTAMP": pd.to_datetime,
    "STATUS": str,
    "REPORT": str,
},
"FinanceBankingVI": {
    "Companycode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "InterestDiscountOnAdvancesBills": float,
    "IncomeFromInvestments": float,
    "InterestOnBalanceWithRBIAndOtherInterBankFunds": float,
    "Others": float,
    "TotalInterestEarned": float,
    "OtherIncome": float,
    "TotalIncome": float,
    "InterestExpended": float,
    "PaymentsToAndProvisionsForEmployees": float,
    "Depreciation": float,
    "DepreciationOnLeasedAssets": float,
    "OperatingExpensesExcludesEmployeeCostAndDepreciation": float,
    "TotalOperatingExpenses": float,
    "ProvisionTowardsIncomeTax": float,
    "ProvisionTowardsDeferredTax": float,
    "ProvisionTowardsOtherTaxes": float,
    "OtherProvisionsAndContingencies": float,
    "TotalProvisionsAndContingencies": float,
    "TotalExpenditure": float,
    "NetProfitLossForTheYear": float,
    "PriorPeriodItems": float,
    "ExtraordinaryItems": float,
    "NetProfitLossForTheYearAfterEI": float,
    "ProfitLossBroughtForward": float,
    "TransferredOnAmalgamation": float,
    "TotalProfitLoss": float,
    "TransferToFromStatutoryReserve": float,
    "TransferToFromReserveFund": float,
    "TransferToFromSpecialReserve": float,
    "TransferToFromCapitalReserve": float,
    "TransferToFromGeneralReserve": float,
    "TransferToFromInvestmentReserve": float,
    "TransferToFromRevenueAndOtherReserves": float,
    "DividendForThePreviousYear": float,
    "EquityShareDividend": float,
    "PreferenceShareDividend": float,
    "TaxOnDividend": float,
    "BalanceCarriedOverToBalanceSheet": float,
    "TotalAppropriations": float,
    "EquityShares": float,
    "EquityCapital": float,
    "PreferenceCapital": float,
    "TotalShareCapital": float,
    "RevaluationReserves": float,
    "ReservesAndSurplus": float,
    "TotalReservesAndSurplus": float,
    "MoneyAgainstShareWarrants": float,
    "EmployeesStockOptions": float,
    "TotalShareHoldersFunds": float,
    "EquityShareApplicationMoney": float,
    "PrefShareApplicationMoney": float,
    "ShareCapitalSuspense": float,
    "Deposits": float,
    "Borrowings": float,
    "OtherLiabilitiesAndProvisions": float,
    "TotalCapitalAndLiabilities": float,
    "CashAndBalancesWithReserveBankOfIndia": float,
    "BalancesWithBanksMoneyAtCallAndShortNotice": float,
    "Investments": float,
    "Advances": float,
    "FixedAssets": float,
    "OtherAssets": float,
    "TotalAssets": float,
    "NetProfitLossBeforeExtraordinaryItemsAndTax": float,
    "NetCashFlowFromOperatingActivities": float,
    "NetCashUsedInInvestingActivities": float,
    "NetCashUsedFromFinancingActivities": float,
    "ForeignExchangeGainsLosses": float,
    "NetIncDecInCashAndCashEquivalents": float,
    "CashAndCashEquivalentBeginOfYear": float,
    "CashAndCashEquivalentEndOfYear": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "NumberOfBranches": float,
    "NumberOfEmployees": float,
    "CapitalAdequacyRatios": float,
    "KeyPerformanceTier1": float,
    "KeyPerformanceTier2": float,
    "GrossNPARs": float,
    "GrossNPAPercentage": float,
    "NetNPARs": float,
    "NetNPAPercentage": float,
    "NetNPAToAdvancesPercentage": float,
    "BillsForCollection": float,
    "ContingentLiabilities": float,
    "EquityDividendRate": float,
    "BonusEquityShareCapital": float,
    "ModifiedDate": pd.to_datetime,
},
"FinanceConsolidatedBankingVI": {
    "Companycode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "InterestDiscountOnAdvancesBills": float,
    "IncomeFromInvestments": float,
    "InterestOnBalanceWithRBIAndOtherInterBankFunds": float,
    "Others": float,
    "TotalInterestEarned": float,
    "OtherIncome": float,
    "TotalIncome": float,
    "InterestExpended": float,
    "PaymentsToAndProvisionsForEmployees": float,
    "Depreciation": float,
    "DepreciationOnLeasedAssets": float,
    "OperatingExpensesExcludesEmployeeCostAndDepreciation": float,
    "TotalOperatingExpenses": float,
    "ProvisionTowardsIncomeTax": float,
    "ProvisionTowardsDeferredTax": float,
    "ProvisionTowardsOtherTaxes": float,
    "OtherProvisionsAndContingencies": float,
    "TotalProvisionsAndContingencies": float,
    "TotalExpenditure": float,
    "NetProfitLossForTheYear": float,
    "PriorPeriodItems": float,
    "ExtraordinaryItems": float,
    "NetProfitLossForTheYearAfterEI": float,
    "IncomeMinorityInterest": float,
    "ShareOfProfitLossFromAssociate": float,
    "ProfitLossAfterMIAndAssociates": float,
    "ProfitLossBroughtForward": float,
    "TransferredOnAmalgamation": float,
    "TotalProfitLoss": float,
    "TransferToFromStatutoryReserve": float,
    "TransferToFromReserveFund": float,
    "TransferToFromSpecialReserve": float,
    "TransferToFromCapitalReserve": float,
    "TransferToFromGeneralReserve": float,
    "TransferToFromInvestmentReserve": float,
    "TransferToFromRevenueAndOtherReserves": float,
    "DividendForThePreviousYear": float,
    "EquityShareDividend": float,
    "PreferenceShareDividend": float,
    "TaxOnDividend": float,
    "BalanceCarriedOverToBalanceSheet": float,
    "TotalAppropriations": float,
    "EquityShares": float,
    "EquityCapital": float,
    "PreferenceCapital": float,
    "TotalShareCapital": float,
    "RevaluationReserves": float,
    "ReservesAndSurplus": float,
    "TotalReservesAndSurplus": float,
    "MoneyAgainstShareWarrants": float,
    "EmployeesStockOptions": float,
    "TotalShareHoldersFunds": float,
    "EquityShareApplicationMoney": float,
    "PrefShareApplicationMoney": float,
    "ShareCapitalSuspense": float,
    "LiabilityMinorityInterest": float,
    "PolicyHoldersFunds": float,
    "LiabilityGroupShareInJointVentures": float,
    "Deposits": float,
    "Borrowings": float,
    "OtherLiabilitiesAndProvisions": float,
    "TotalCapitalAndLiabilities": float,
    "CashAndBalancesWithReserveBankOfIndia": float,
    "BalancesWithBanksMoneyAtCallAndShortNotice": float,
    "Investments": float,
    "Advances": float,
    "FixedAssets": float,
    "OtherAssets": float,
    "AssetMinorityInterest": float,
    "AssetGroupShareInJointVentures": float,
    "TotalAssets": float,
    "NetProfitLossBeforeExtraordinaryItemsAndTax": float,
    "NetCashFlowFromOperatingActivities": float,
    "NetCashUsedInInvestingActivities": float,
    "NetCashUsedFromFinancingActivities": float,
    "ForeignExchangeGainsLosses": float,
    "NetIncDecInCashAndCashEquivalents": float,
    "CashAndCashEquivalentBeginOfYear": float,
    "CashAndCashEquivalentEndOfYear": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "NumberOfBranches": float,
    "NumberOfEmployees": float,
    "CapitalAdequacyRatios": float,
    "KeyPerformanceTier1": float,
    "KeyPerformanceTier2": float,
    "GrossNPARs": float,
    "GrossNPAPercentage": float,
    "NetNPARs": float,
    "NetNPAPercentage": float,
    "NetNPAToAdvancesPercentage": float,
    "BillsForCollection": float,
    "ContingentLiabilities": float,
    "BonusEquityShareCapital": float,
    "ModifiedDate": pd.to_datetime,
},
"FinanceConsolidatedNonBankingVI": {
    "Companycode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "RevenueFromOperationsGross": float,
    "LessExciseSeviceTaxOtherLevies": float,
    "RevenueFromOperationsNet": float,
    "OtherOperatingRevenues": float,
    "TotalOperatingRevenues": float,
    "OtherIncome": float,
    "GroupShareInJointVenturesInIncome": float,
    "TotalRevenue": float,
    "CostOfMaterialsConsumed": float,
    "PurchaseOfStockInTrade": float,
    "PurchaseOfCrudeOilAndOthers": float,
    "CostOfPowerPurchased": float,
    "CostOfFuel": float,
    "AircraftFuelExpenses": float,
    "AircraftLeaseRentals": float,
    "OperatingAndDirectExpenses": float,
    "ChangesIninventoriesOfFGWIPAndStockInTrade": float,
    "EmployeeBenefitExpenses": float,
    "FinanceCosts": float,
    "ProvisionsAndContingencies": float,
    "DepreciationAndAmortisationExpenses": float,
    "MiscExpensesWOff": float,
    "OtherExpenses": float,
    "LessInterUnitSegmentDivisionTransfer": float,
    "LessTransferToFromInvestmentFixedAssetsOthers": float,
    "LessAmountsTransferToCapitalAccounts": float,
    "LessShareOfLossFromPartnershipFirm": float,
    "GroupShareInJointVenturesInExpenditure": float,
    "TotalExpenses": float,
    "ProfitLossBeforeExceptionalExtraOrdinaryItemsAndTax": float,
    "ExceptionalItems": float,
    "ProfitLossBeforeTax": float,
    "CurrentTax": float,
    "LessMATCredit": float,
    "DeferredTax": float,
    "OtherDirectTaxes": float,
    "TaxForEarlierYears": float,
    "TotalTaxExpensesContinuedOperations": float,
    "ProfitLossAfterTaxAndBeforeExtraOrdinaryItems": float,
    "PriorPeriodItems": float,
    "ExtraordinaryItems": float,
    "ProfitLossFromContinuingOperations": float,
    "ProfitLossFromDiscontinuingOperations": float,
    "TotalTaxExpensesDiscontinuingOperations": float,
    "NetProfitLossFromDiscontinuingOperations": float,
    "ProfitLossForThePeriod": float,
    "IncomeMinorityInterest": float,
    "ShareOfProfitLossOfAssociates": float,
    "ProfitLossAfterMIAndAssociates": float,
    "EquityShares": float,
    "EquityCapital": float,
    "PreferenceCapital": float,
    "TotalShareCapital": float,
    "RevaluationReserves": float,
    "ReservesAndSurplus": float,
    "TotalReservesAndSurplus": float,
    "MoneyAgainstShareWarrants": float,
    "EmployeesStockOptions": float,
    "TotalShareHoldersFunds": float,
    "PrefSharesIssuedBySubsidiaryCos": float,
    "EquityShareApplicationMoney": float,
    "PreferenceShareApplicationMoney": float,
    "ShareCapitalSuspense": float,
    "HybridDebtOtherSecurities": float,
    "StatutoryConsumerReserves": float,
    "SpecialApprnTowardsProjectCost": float,
    "ServiceLineContributionFromConsumers": float,
    "GovernmentOtherGrants": float,
    "LiabilityMinorityInterest": float,
    "PolicyHoldersFunds": float,
    "LiabilityGroupShareInJointVentures": float,
    "LongTermBorrowings": float,
    "DeferredTaxLiabilities": float,
    "OtherLongTermLiabilities": float,
    "LongTermProvisions": float,
    "TotalNonCurrentLiabilities": float,
    "ForeignCurrencyMonetaryItemTranslationDifferenceLiabilityAccoun": float,
    "ShortTermBorrowings": float,
    "TradePayables": float,
    "OtherCurrentLiabilities": float,
    "ShortTermProvisions": float,
    "TotalCurrentLiabilities": float,
    "TotalCapitalAndLiabilities": float,
    "TangibleAssets": float,
    "IntangibleAssets": float,
    "CapitalWorkInProgress": float,
    "IntangibleAssetsUnderDevelopment": float,
    "OtherAssets": float,
    "NonCurrentInvestmentsUnquotedBookValue": float,
    "ConstructionStores": float,
    "MiningDevelopmentExpenditure": float,
    "AssetsHeldForSale": float,
    "TotalFixedAssets": float,
    "GoodwillOnConsolidation": float,
    "NonCurrentInvestments": float,
    "DeferredTaxAssets": float,
    "LongTermLoansAndAdvances": float,
    "OtherNonCurrentAssets": float,
    "TotalNonCurrentAssets": float,
    "AssetMinorityInterest": float,
    "AssetGroupShareInJointVentures": float,
    "ForeignCurrencyMonetaryItemTranslationDiffAcct": float,
    "CurrentInvestments": float,
    "Inventories": float,
    "TradeReceivables": float,
    "CashAndCashEquivalents": float,
    "ShortTermLoansAndAdvances": float,
    "OtherCurrentAssets": float,
    "TotalCurrentAssets": float,
    "TotalAssets": float,
    "NetProfitLossBeforeExtraordinaryItemsAndTax": float,
    "NetCashFlowFromOperatingActivities": float,
    "NetCashUsedInInvestingActivities": float,
    "NetCashUsedFromFinancingActivities": float,
    "ForeignExchangeGainsLosses": float,
    "AdjustmentsOnAmalgamationMergerDemergerOthers": float,
    "NetIncDecInCashAndCashEquivalents": float,
    "CashAndCashEquivalentBeginOfYear": float,
    "CashAndCashEquivalentEndOfYear": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "ContingentLiabilities": float,
    "CIFValueOfRawMaterials": float,
    "CIFValueOfStoresSparesAndLooseTools": float,
    "CIFValueOfOtherGoods": float,
    "CIFValueOfCapitalGoods": float,
    "ExpenditureInForeignCurrency": float,
    "DividendRemittanceInForeignCurrency": float,
    "ForeignExchangeFOB": float,
    "ForeignExchangeOtherEarnings": float,
    "ImportedRawMaterials": float,
    "IndigenousRawMaterials": float,
    "ImportedStoresAndSpares": float,
    "IndigenousStoresAndSpares": float,
    "EquityShareDividend": float,
    "PreferenceShareDividend": float,
    "TaxOnDividend": float,
    "BonusEquityShareCapital": float,
    "NonCurrentInvestmentsQuotedMarketValue": float,
    "CurrentInvestmentsQuotedMarketValue": float,
    "CurrentInvestmentsUnquotedBookValue": float,
    "ModifiedDate": pd.to_datetime,
},
"FinanceNonBankingVI": {
    "Companycode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "RevenueFromOperationsGross": float,
    "LessExciseSeviceTaxOtherLevies": float,
    "RevenueFromOperationsNet": float,
    "OtherOperatingRevenues": float,
    "TotalOperatingRevenues": float,
    "OtherIncome": float,
    "TotalRevenue": float,
    "CostOfMaterialsConsumed": float,
    "PurchaseOfStockInTrade": float,
    "PurchaseOfCrudeOilAndOthers": float,
    "CostOfPowerPurchased": float,
    "CostOfFuel": float,
    "AircraftFuelExpenses": float,
    "AircraftLeaseRentals": float,
    "OperatingAndDirectExpenses": float,
    "ChangesIninventoriesOfFGWIPAndStockInTrade": float,
    "EmployeeBenefitExpenses": float,
    "FinanceCosts": float,
    "ProvisionsAndContingencies": float,
    "DepreciationAndAmortisationExpenses": float,
    "MiscExpensesWOff": float,
    "OtherExpenses": float,
    "LessInterUnitSegmentDivisionTransfer": float,
    "LessTransferToFromInvestmentFixedAssetsOthers": float,
    "LessAmountsTransferToCapitalAccounts": float,
    "LessShareOfLossFromPartnershipFirm": float,
    "TotalExpenses": float,
    "ProfitLossBeforeExceptionalExtraOrdinaryItemsAndTax": float,
    "ExceptionalItems": float,
    "ProfitLossBeforeTax": float,
    "CurrentTax": float,
    "LessMATCredit": float,
    "DeferredTax": float,
    "OtherDirectTaxes": float,
    "TaxForEarlierYears": float,
    "TotalTaxExpensesContinuedOperations": float,
    "ProfitLossAfterTaxAndBeforeExtraOrdinaryItems": float,
    "PriorPeriodItems": float,
    "ExtraordinaryItems": float,
    "ProfitLossFromContinuingOperations": float,
    "ProfitLossFromDiscontinuingOperations": float,
    "TotalTaxExpensesDiscontinuingOperations": float,
    "NetProfitLossFromDiscontinuingOperations": float,
    "ProfitLossForThePeriod": float,
    "EquityShares": float,
    "EquityCapital": float,
    "PreferenceCapital": float,
    "TotalShareCapital": float,
    "RevaluationReserves": float,
    "ReservesAndSurplus": float,
    "TotalReservesAndSurplus": float,
    "MoneyAgainstShareWarrants": float,
    "EmployeesStockOptions": float,
    "TotalShareHoldersFunds": float,
    "EquityShareApplicationMoney": float,
    "PreferenceShareApplicationMoney": float,
    "ShareCapitalSuspense": float,
    "HybridDebtOtherSecurities": float,
    "StatutoryConsumerReserves": float,
    "SpecialApprnTowardsProjectCost": float,
    "ServiceLineContributionFromConsumers": float,
    "GovernmentOtherGrants": float,
    "LongTermBorrowings": float,
    "DeferredTaxLiabilities": float,
    "OtherLongTermLiabilities": float,
    "LongTermProvisions": float,
    "TotalNonCurrentLiabilities": float,
    "ForeignCurrencyMonetaryItemTranslationDifferenceLiabilityAccoun": float,
    "ShortTermBorrowings": float,
    "TradePayables": float,
    "OtherCurrentLiabilities": float,
    "ShortTermProvisions": float,
    "TotalCurrentLiabilities": float,
    "TotalCapitalAndLiabilities": float,
    "TangibleAssets": float,
    "IntangibleAssets": float,
    "CapitalWorkInProgress": float,
    "IntangibleAssetsUnderDevelopment": float,
    "OtherAssets": float,
    "ConstructionStores": float,
    "MiningDevelopmentExpenditure": float,
    "AssetsHeldForSale": float,
    "TotalFixedAssets": float,
    "NonCurrentInvestments": float,
    "DeferredTaxAssets": float,
    "LongTermLoansAndAdvances": float,
    "OtherNonCurrentAssets": float,
    "TotalNonCurrentAssets": float,
    "ForeignCurrencyMonetaryItemTranslationDiffAcct": float,
    "CurrentInvestments": float,
    "Inventories": float,
    "TradeReceivables": float,
    "CashAndCashEquivalents": float,
    "ShortTermLoansAndAdvances": float,
    "OtherCurrentAssets": float,
    "TotalCurrentAssets": float,
    "TotalAssets": float,
    "NetProfitLossBeforeExtraordinaryItemsAndTax": float,
    "NetCashFlowFromOperatingActivities": float,
    "NetCashUsedInInvestingActivities": float,
    "NetCashUsedFromFinancingActivities": float,
    "ForeignExchangeGainsLosses": float,
    "AdjustmentsOnAmalgamationMergerDemergerOthers": float,
    "NetIncDecInCashAndCashEquivalents": float,
    "CashAndCashEquivalentBeginOfYear": float,
    "CashAndCashEquivalentEndOfYear": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "ContingentLiabilities": float,
    "CIFValueOfRawMaterials": float,
    "CIFValueOfStoresSparesAndLooseTools": float,
    "CIFValueOfOtherGoods": float,
    "CIFValueOfCapitalGoods": float,
    "ExpenditureInForeignCurrency": float,
    "DividendRemittanceInForeignCurrency": float,
    "ForeignExchangeFOB": float,
    "ForeignExchangeOtherEarnings": float,
    "ImportedRawMaterials": float,
    "IndigenousRawMaterials": float,
    "ImportedStoresAndSpares": float,
    "IndigenousStoresAndSpares": float,
    "EquityShareDividend": float,
    "PreferenceShareDividend": float,
    "TaxOnDividend": float,
    "EquityDividendRate": float,
    "BonusEquityShareCapital": float,
    "NonCurrentInvestmentsQuotedMarketValue": float,
    "NonCurrentInvestmentsUnquotedBookValue": float,
    "CurrentInvestmentsQuotedMarketValue": float,
    "CurrentInvestmentsUnquotedBookValue": float,
    "ModifiedDate": pd.to_datetime,
},
"FundMaster": {
    "CompanyCode": float,
    "CompanyName": str,
    "SponsorName": str,
    "TrusteeCompanyName": str,
    "MFSetUpDate": pd.to_datetime,
    "AMCCode": float,
    "AMCName": str,
    "AMCIncDate": pd.to_datetime,
    "CEOName": str,
    "CIOName": str,
    "FundManagerName": str,
    "ComplianceOfficerName": str,
    "InvServiceOfficerName": str,
    "AuditorsName": str,
    "CustodianName": str,
    "RegistrarName": str,
    "TypeOfMutualFund": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"HalfYearlyResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Half": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "PrioryearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Percentage_OfGrossNPA": float,
    "Percentage_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "BeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"IndexHistory": {
    "TICKER": str,
    "DATE": pd.to_datetime,
    "OPEN": float,
    "HIGH": float,
    "LOW": float,
    "CLOSE": float,
    "VOL": float,
},
"IndexHistory_DUPLICATED": {
    "TICKER": str,
    "DATE": pd.to_datetime,
    "OPEN": float,
    "HIGH": float,
    "LOW": float,
    "CLOSE": float,
    "VOL": float,
},
"IndexOHLC": {
    "IndexName": str,
    "NSECode": str,
    "Date": pd.to_datetime,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Points Change": float,
    "Change": float,
    "Volume": float,
    "Turnover": float,
    "PE": float,
    "PB": float,
    "Div Yield": float,
},
"IndexOHLC_DUPLICATED": {
    "IndexName": str,
    "NSECode": str,
    "Date": pd.to_datetime,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Points Change": float,
    "Change": float,
    "Volume": float,
    "Turnover": float,
    "PE": float,
    "PB": float,
    "Div Yield": float,
},
"IndividualHolding": {
    "CompanyCode": float,
    "ShareHoldingDate": pd.to_datetime,
    "InstrumentDescription": str,
    "Name": str,
    "NoOfShares": float,
    "PromoterFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"IndustryDivisor": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"SubIndustryDivisor": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"IndustryIndexList": {
    "GenDate": pd.to_datetime,
    "IndustryIndexName": str,
    "Industry": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"SubIndustryIndexList": {
    "GenDate": pd.to_datetime,
    "SubIndustryIndexName": str,
    "SubIndustry": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"IndustryList": {
    "CompanyCode": float,
    "CompanyName": str,
    "NSECode": str,
    "BSECode": int,
    "ISIN": str,
    "Industry": str,
    "Sector": str,
    "SubSector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PrevClose": float,
    "Change": float,
    "OS": float,
    "FaceValue": float,
    "FreeFloat": float,
    "GenDate": pd.to_datetime,
    "FF_Open": float,
    "FF_High": float,
    "FF_Low": float,
    "FF_Close": float,
    "IndustryIndexName": str,
    "SectorIndexName": str,
    "SubSectorIndexName": str,
    "PAT": float,
    "Equity": float,
    "OS_Close": float,
    "prev_pat": float,
    "prev_equity": float,
},
"IndustryList_DUPLICATED": {
    "CompanyCode": float,
    "CompanyName": str,
    "NSECode": str,
    "BSECode": int,
    "ISIN": str,
    "Industry": str,
    "Sector": str,
    "SubSector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PrevClose": float,
    "Change": float,
    "OS": float,
    "FaceValue": float,
    "FreeFloat": float,
    "GenDate": pd.to_datetime,
    "FF_Open": float,
    "FF_High": float,
    "FF_Low": float,
    "FF_Close": float,
    "IndustryIndexName": str,
    "SectorIndexName": str,
    "SubSectorIndexName": str,
    "PAT": float,
    "Equity": float,
    "OS_Close": float,
    "prev_pat": float,
    "prev_equity": float,
},
"IndustryMapping": {
    "IndustryCode": int,
    "IndustryName": str,
    "BroadIndustryCode": int,
    "BroadIndustryName": str,
    "Industry": str,
    "Code": str,
    "SubSector": str,
    "SubSectorCode": str,
    "Sector": str,
    "SectorCode": str,
    "SectorIndexName": str,
    "SubSectorIndexName": str,
    "IndustryIndexName": str,
},
"IndustryMaster": {
    "IndustryCode": float,
    "IndustryName": str,
    "BroadIndustryCode": float,
    "BroadIndustryName": str,
    "MajorSectorCode": float,
    "MajorSectorName": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"KeyExecutives": {
    "Companycode": float,
    "TitleDescription": str,
    "ExecutiveName": str,
    "DesignationDescription": str,
    "ModifiedDate": pd.to_datetime,
    "RowId": int,
},
"Locations": {
    "CompanyCode": float,
    "AddressType": str,
    "Address": str,
    "Address2": str,
    "Address3": str,
    "Address4": str,
    "CityName": str,
    "StateName": str,
    "CountryName": str,
    "Pincode": str,
    "TelephoneNumbers": str,
    "Phone2": str,
    "Phone3": str,
    "Phone4": str,
    "Telex": str,
    "Telex1": str,
    "Fax": str,
    "Fax1": str,
    "Grams": str,
    "EMailID": str,
    "WebSite": str,
    "ModifiedDate": pd.to_datetime,
},
"MFInvestments": {
    "MFDate": pd.to_datetime,
    "InstrumentType": str,
    "Purchase": float,
    "Sales": float,
    "NetInvestment": float,
    "ModifiedDate": pd.to_datetime,
},
"MFMergeList": {
    "SchemeCode": float,
    "SchemeName": str,
    "SchemePlanCode": int,
    "SchemeCategoryDescription": str,
    "MainCategory": str,
    "SchemeTypeDescription": str,
    "AUM": float,
    "HoldingDate": pd.to_datetime,
    "InvestedCompanyCode": float,
    "InvestedCompanyName": str,
    "Industry": str,
    "Quantity": float,
    "Percentage": float,
    "ISINCode": str,
    "Market Cap Class": str,
    "GenDate": pd.to_datetime,
    "Market Rank": float,
},
"ManagementTeam": {
    "Companycode": float,
    "TitleDescription": str,
    "PersonName": str,
    "DesignationDescription": str,
    "ModifiedDate": pd.to_datetime,
    "RowId": int,
},
"NSE": {
    "SYMBOL": str,
    "SERIES": str,
    "OPEN": float,
    "HIGH": float,
    "LOW": float,
    "CLOSE": float,
    "LAST": float,
    "PREVCLOSE": float,
    "TOTTRDQTY": float,
    "TOTTRDVAL": float,
    "TIMESTAMP": pd.to_datetime,
    "TOTALTRADES": float,
    "ISIN": str,
},
"News": {
    "CompanyCode": float,
    "NewsDate": pd.to_datetime,
    "Headlines": str,
    "SourceName": str,
    "NewsTypeDescription": str,
    "NewsClassfication": str,
    "Synopsis": str,
    "ModifiedDate": pd.to_datetime,
},
"NinemonthsResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Nine": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestements": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "PrioryearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Percentage_OfGrossNPA": float,
    "Percentage_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "BeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"OHLC": {
    "Company": str,
    "NSECode": str,
    "BSECode": int,
    "ISIN": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Date": pd.to_datetime,
    "Value": float,
    "Volume": float,
    "CompanyCode": float,
},
"OHLC_DUPLICATED": {
    "Company": str,
    "NSECode": str,
    "BSECode": int,
    "ISIN": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Date": pd.to_datetime,
    "Value": float,
    "Volume": int,
    "CompanyCode": float,
},
"PE": {
    "CompanyCode": float,
    "NSECode": str,
    "BSECode": int,
    "ISINCode": str,
    "Market Cap Value": float,
    "Market Cap Rank": int,
    "Market Cap Class": str,
    "PE": float,
    "PE High": float,
    "PE High Date": pd.to_datetime,
    "PE Low": float,
    "PE Low Date": pd.to_datetime,
    "GenDate": pd.to_datetime,
},
"PE_DUPLICATED": {
    "CompanyCode": float,
    "NSECode": str,
    "BSECode": int,
    "ISINCode": str,
    "Market Cap Value": float,
    "Market Cap Rank": int,
    "Market Cap Class": str,
    "PE": float,
    "PE High": float,
    "PE High Date": pd.to_datetime,
    "PE Low": float,
    "PE Low Date": pd.to_datetime,
    "GenDate": pd.to_datetime,
},
"PledgeShares": {
    "CompanyCode": float,
    "SHPDate": pd.to_datetime,
    "PledgeShares_Nos": int,
    "PledgeShares_Pct": float,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"Products": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "NoOfMonths": int,
    "ProductCode": float,
    "ProductName": str,
    "LicensedCapacity": float,
    "LICUOM": str,
    "InstalledCapacity": float,
    "InstalledUOM": str,
    "ProductionQuantity": float,
    "ProductionUOM": str,
    "SalesQuantity": float,
    "SalesUOM": str,
    "SalesValue": float,
    "OpeningQuantity": float,
    "OpeningUOM": str,
    "OpeningValue": float,
    "ClosingQuantity": float,
    "ClosingUOM": str,
    "ClosingValue": float,
    "BoughtOutQuantity": float,
    "BroughtOutUOM": str,
    "ProductMix": float,
    "ModifiedDate": pd.to_datetime,
},
"QuarterlyEPS": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PATRAW": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
    "Ext_Flag": bool,
    "Q1 EPS Growth": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
},
"QuarterlyEPS_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PATRAW": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
    "Ext_Flag": bool,
    "Q1 EPS Growth": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
},
"QuarterlyEPS_ToBeDeleted": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PATRAW": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
    "Ext_Flag": bool,
    "Q1 EPS Growth": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
},
"QuarterlyResults": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestments": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "PrioryearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Percentage_OfGrossNPA": float,
    "Percentage_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "BeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"QuarterlyResults_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestments": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "PrioryearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Percentage_OfGrossNPA": float,
    "Percentage_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "BeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"QuarterlyResults_TEMP": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "OperatingIncome": float,
    "OtherOperatingIncome": float,
    "TotalIncomeFromOperations": float,
    "IntOrDiscOnAdvOrBills": float,
    "IncomeOnInvestments": float,
    "IntOnBalancesWithRBI": float,
    "Others": float,
    "OtherRecurringIncome": float,
    "StockAdjustment": float,
    "RawMaterialConsumed": float,
    "PurchaseOfTradedGoods": float,
    "PowerAndFuel": float,
    "EmployeeExpenses": float,
    "Excise": float,
    "AdminAndSellingExpenses": float,
    "ResearchAndDevelopmentExpenses": float,
    "ExpensesCapitalised": float,
    "OtherExpeses": float,
    "PL_Before_OtherInc_Int_Excpltem_Tax": float,
    "PL_Before_Int_ExcpItem_Tax": float,
    "InterestCharges": float,
    "PL_Before_ExcpItem_Tax": float,
    "ExceptionalItems": float,
    "Depreciation": float,
    "OperatingProfitBeforeProvisionsAndContingencies": float,
    "BankProvisionsMade": float,
    "PL_Before_Tax": float,
    "TaxCharges": float,
    "PL_After_TaxFromOrdineryActivities": float,
    "ExtraOrdinaryItem": float,
    "ReportedPAT": float,
    "PrioryearAdj": float,
    "ReservesWrittenBack": float,
    "EquityCapital": float,
    "ReservesAndSurplus": float,
    "EqDividendRate": float,
    "AggregateOfNonPromotoNoOfShares": float,
    "AggregateOfNonPromotoHoldingPercent": float,
    "GovernmentShare": float,
    "CapitalAdequacyRatio": float,
    "CapitalAdequacyBasell": float,
    "GrossNPA": float,
    "NetNPA": float,
    "Percentage_OfGrossNPA": float,
    "Percentage_OfNetNPA": float,
    "ReturnOnAssets_Per": float,
    "BeforeBasicEPS": float,
    "BeforeDilutedEPS": float,
    "AfterBasicEPS": float,
    "AfterDilutedEPS": float,
    "En_NumberOfShares": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "En_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "NonEn_NumberOfShares": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Hol_of_Pro_And_Group": float,
    "NonEn_Per_Share_As_Per_Of_Tot_Sh_Cap_of_Company": float,
    "ModifiedDate": pd.to_datetime,
},
"RatiosBankingVI": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "CashEPS": float,
    "BVPerShareExclRevalReserve": float,
    "BVPerShareInclRevalReserve": float,
    "DividendPerShare": float,
    "OperatingRevenuePerShare": float,
    "NPPerShare": float,
    "InterestIncomePerEmployee": float,
    "NPPerEmployee": float,
    "BusinessPerEmployee": float,
    "InterestIncomePerBranch": float,
    "NPPerBranches": float,
    "BusinessPerBranches": float,
    "NPM": float,
    "OPM": float,
    "ROA": float,
    "ROE": float,
    "NIM": float,
    "CostToIncome": float,
    "InterestIncomeByEarningAssets": float,
    "NonInterestIncomeByEarningAssets": float,
    "OperatingProfitByEarningAssets": float,
    "OperatingExpensesByEarningAssets": float,
    "InterestExpensesByEarningAssets": float,
    "EnterpriseValue": float,
    "EVPerNetSales": float,
    "PriceToBV": float,
    "PriceToSales": float,
    "RetentionRatios": float,
    "EarningsYield": float,
    "ModifiedDate": pd.to_datetime,
},
"RatiosConslidatedNonBankingVI": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "CashEPS": float,
    "BVPerShareExclRevalReserve": float,
    "BVPerShareInclRevalReserve": float,
    "OperatingRevenuePerShare": float,
    "PBDITPerShare": float,
    "PBITPerShare": float,
    "PBTPerShare": float,
    "NPPerShare": float,
    "NPAfterMIPerShare": float,
    "PBDITMargin": float,
    "PBITMargin": float,
    "PBTMargin": float,
    "NPMargin": float,
    "NPAfterMIMargin": float,
    "RONW": float,
    "ROCE": float,
    "ReturnOnAssets": float,
    "LongTermDebtEquity": float,
    "DebtEquity": float,
    "AssetTurnover": float,
    "CurrentRatio": float,
    "QuickRatio": float,
    "InventoryTurnoverRatio": float,
    "DividendPayoutNP": float,
    "DividendPayoutCP": float,
    "EarningRetention": float,
    "CashEarningRetention": float,
    "InterestCoverage": float,
    "InterestCoveragePostTax": float,
    "EnterpriseValue": float,
    "EVPerNetSales": float,
    "EVPerEBITDA": float,
    "MarketCapPerSales": float,
    "RetentionRatios": float,
    "PriceToBV": float,
    "PriceToSales": float,
    "EarningsYield": float,
    "ModifiedDate": pd.to_datetime,
},
"RatiosConsolidatedBankingVI": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "CashEPS": float,
    "BVPerShareExclRevalReserve": float,
    "BVPerShareInclRevalReserve": float,
    "DividendPerShare": float,
    "OperatingRevenuePerShare": float,
    "NPPerShare": float,
    "NPAfterMIPerShare": float,
    "InterestIncomePerEmployee": float,
    "NPPerEmployee": float,
    "NPAfterMIPerEmployee": float,
    "BusinessPerEmployee": float,
    "InterestIncomePerBranch": float,
    "NPPerBranches": float,
    "NPAfterMIPerBranches": float,
    "BusinessPerBranches": float,
    "NPM": float,
    "NPMAfterMI": float,
    "OPM": float,
    "ROA": float,
    "ROE": float,
    "NIM": float,
    "CostToIncome": float,
    "InterestIncomeByEarningAssets": float,
    "NonInterestIncomeByEarningAssets": float,
    "OperatingProfitByEarningAssets": float,
    "OperatingExpensesByEarningAssets": float,
    "InterestExpensesByEarningAssets": float,
    "EnterpriseValue": float,
    "EVPerNetSales": float,
    "PriceToBV": float,
    "PriceToSales": float,
    "RetentionRatios": float,
    "EarningsYield": float,
    "ModifiedDate": pd.to_datetime,
},
"RatiosMergeList": {
    "CompanyCode": float,
    "CompanyName": str,
    "Industry": str,
    "ROEYearEnding": pd.to_datetime,
    "Months": int,
    "FaceValue": float,
    "SalesGrowth": float,
    "NPM": float,
    "ROE": float,
    "Debt": float,
    "TTMYearEnding": pd.to_datetime,
    "GenDate": pd.to_datetime,
},
"RatiosMergeList_DUPLICATED": {
    "CompanyCode": float,
    "CompanyName": str,
    "Industry": str,
    "ROEYearEnding": pd.to_datetime,
    "Months": int,
    "FaceValue": float,
    "SalesGrowth": float,
    "NPM": float,
    "ROE": float,
    "Debt": float,
    "TTMYearEnding": pd.to_datetime,
    "GenDate": pd.to_datetime,
},
"RatiosNonBankingVI": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Type": str,
    "FaceValue": float,
    "BasicEPS": float,
    "DilutedEPS": float,
    "CashEPS": float,
    "BVPerShareExclRevalReserve": float,
    "BVPerShareInclRevalReserve": float,
    "DividendPerShare": float,
    "OperatingRevenuePerShare": float,
    "PBDITPerShare": float,
    "PBITPerShare": float,
    "PBTPerShare": float,
    "NPperShare": float,
    "PBDITMargin": float,
    "PBITMargin": float,
    "PBTMargin": float,
    "NPMargin": float,
    "RONW": float,
    "ROCE": float,
    "ReturnOnAssets": float,
    "LongTermDebtEquity": float,
    "DebtEquity": float,
    "AssetTurnover": float,
    "CurrentRatio": float,
    "QuickRatio": float,
    "InventoryTurnoverRatio": float,
    "DividendPayoutNP": float,
    "DividendPayoutCP": float,
    "EarningRetention": float,
    "CashEarningRetention": float,
    "InterestCoverage": float,
    "InterestCoveragePostTax": float,
    "EnterpriseValue": float,
    "EVPerNetSales": float,
    "EVPerEBITDA": float,
    "MarketCapPerSales": float,
    "RetentionRatios": float,
    "PriceToBV": float,
    "PriceToSales": float,
    "EarningsYield": float,
    "ModifiedDate": pd.to_datetime,
},
"RawMaterials": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "ProductCode": float,
    "ProductName": str,
    "RawMatUOM": str,
    "RawMatQuantity": float,
    "RawMatValue": float,
    "ProdMix": float,
    "ModifiedDate": pd.to_datetime,
},
"Registrars": {
    "CompanyCode": float,
    "AgencyName": str,
    "Address": str,
    "Address2": str,
    "Address3": str,
    "Address4": str,
    "CityName": str,
    "StateName": str,
    "PinCode": str,
    "TelephoneNumbers": str,
    "Phone2": str,
    "Phone3": str,
    "Phone4": str,
    "Fax": str,
    "Telex": str,
    "Grams": str,
    "EmailId": str,
    "Website": str,
    "ModifiedDate": pd.to_datetime,
},
"Rights": {
    "CompanyCode": float,
    "DateOfAnnouncement": pd.to_datetime,
    "ExistingInstrumentType": int,
    "ExistingInstrumentName": str,
    "RatioExisting": float,
    "RatioOffering": float,
    "OfferredInstrumentType": int,
    "OfferredInstrumentName": str,
    "FaceValueExistingInstrument": float,
    "FaceValueOfferedInstrument": float,
    "RightsPremium": float,
    "RecordDate": pd.to_datetime,
    "BookClosureStartDate": pd.to_datetime,
    "BookClosureEndDate": pd.to_datetime,
    "XRDate": pd.to_datetime,
    "Remarks": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"SchemeBoardOfAMC": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "TitleDescription": str,
    "PersonName": str,
    "DesignationDescription": str,
    "ModifyDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemeBoardOfTrustees": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "TitleDescription": str,
    "PersonName": str,
    "DesignationDescription": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemeBonusDetails": {
    "SchemeCode": float,
    "SchemePlanCode": float,
    "BonusDate": pd.to_datetime,
    "Ratio_Offered": str,
    "Ratio_Existing": str,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemeCategoryDetails": {
    "SchemeClassCode": float,
    "SchemeClassDescription": str,
    "CategoryNavDate": pd.to_datetime,
    "NoOfSchemes": int,
    "OneDayReturn": float,
    "OneWeekReturn": float,
    "TwoWeeksReturn": float,
    "OneMonthReturn": float,
    "ThreeMonthsReturn": float,
    "SixMonthsReturn": float,
    "NineMonthsReturn": float,
    "OneYearReturn": float,
    "TwoYearReturn": float,
    "ThreeYearReturn": float,
    "FiveYearReturn": float,
    "InceptionReturn": float,
    "ModifiedDate": pd.to_datetime,
},
"SchemeCorporateDividendDetails": {
    "SchemeCode": float,
    "SchemePlanCode": int,
    "DividendDate": pd.to_datetime,
    "Percentage": float,
    "DividendPerUnit": float,
    "DividendTypeDescription": str,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemeDividendDetails": {
    "SchemeCode": float,
    "SchemePlanCode": float,
    "DividendTypeDescription": str,
    "DividendDate": pd.to_datetime,
    "Percentage": float,
    "DividendPerUnit": float,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemeFundManagerProfile": {
    "SchemeCode": float,
    "SchemePlanCode": float,
    "FundManagerName": str,
    "FMProfile": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeLocations": {
    "EntityCode": float,
    "LocationType": str,
    "Address": str,
    "CityName": str,
    "StateName": str,
    "CountryName": str,
    "Pincode": str,
    "TelephoneNumbers": str,
    "Telex": str,
    "Fax": str,
    "Grams": str,
    "EMailID": str,
    "WebSite": str,
    "ModifiedDate": pd.to_datetime,
},
"SchemeMaster": {
    "SchemeCode": float,
    "SchemeName": str,
    "CompanyCode": float,
    "AMCCode": float,
    "SchemeType": int,
    "SchemeTypeDescription": str,
    "SchemePlanCode": float,
    "SchemePlanDescription": str,
    "SchemeClassCode": int,
    "SchemeClassDescription": str,
    "SchemeCategoryCode": int,
    "SchemeCategoryDescription": str,
    "MainCategory": str,
    "SubCategory": str,
    "LaunchDate": pd.to_datetime,
    "OpenDate": pd.to_datetime,
    "CloseDate": pd.to_datetime,
    "RedemptionDate": pd.to_datetime,
    "MinInvAmnt": float,
    "IntPrice": float,
    "IntPricUOM": int,
    "IntPUOMDes": str,
    "Size": float,
    "SizeUOM": float,
    "SizeUOMDescription": str,
    "OfferPrice": float,
    "AmountRaised": float,
    "AmountRaisedUOM": float,
    "AmountRaisedUOMDescription": str,
    "FundManagerPrefix": str,
    "FundManagerName": str,
    "ListingInformation": bool,
    "EntryLoad": str,
    "ExitLoad": str,
    "RedemptionFerq": str,
    "ISIN_Div_Payout_ISIN_Growth": str,
    "ISIN_Div_Reinvestment": str,
    "RegistrarName": str,
    "EntryLoad_WithSlab_Desc": str,
    "ExitLoad_WithSlab_Desc": str,
    "SIP_Min_Invt": str,
    "SIPEntryLoad": str,
    "SIPExitLoad": str,
    "ExpensesRatio": str,
    "PortFolioTurnOverRatio": str,
    "SWP": str,
    "STP": str,
    "RiskoMeter": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeMasterCopy": {
    "SchemeCode": float,
    "SchemeName": str,
    "CompanyCode": float,
    "AMCCode": float,
    "SchemeType": int,
    "SchemeTypeDescription": str,
    "SchemePlanCode": float,
    "SchemePlanDescription": str,
    "SchemeClassCode": int,
    "SchemeClassDescription": str,
    "SchemeCategoryCode": int,
    "SchemeCategoryDescription": str,
    "MainCategory": str,
    "SubCategory": str,
    "LaunchDate": pd.to_datetime,
    "OpenDate": pd.to_datetime,
    "CloseDate": pd.to_datetime,
    "RedemptionDate": pd.to_datetime,
    "MinInvAmnt": float,
    "IntPrice": float,
    "IntPricUOM": int,
    "IntPUOMDes": str,
    "Size": float,
    "SizeUOM": float,
    "SizeUOMDescription": str,
    "OfferPrice": float,
    "AmountRaised": float,
    "AmountRaisedUOM": float,
    "AmountRaisedUOMDescription": str,
    "FundManagerPrefix": str,
    "FundManagerName": str,
    "ListingInformation": bool,
    "EntryLoad": str,
    "ExitLoad": str,
    "RedemptionFerq": str,
    "ISIN_Div_Payout_ISIN_Growth": str,
    "ISIN_Div_Reinvestment": str,
    "RegistrarName": str,
    "EntryLoad_WithSlab_Desc": str,
    "ExitLoad_WithSlab_Desc": str,
    "SIP_Min_Invt": str,
    "SIPEntryLoad": str,
    "SIPExitLoad": str,
    "ExpensesRatio": str,
    "PortFolioTurnOverRatio": str,
    "SWP": str,
    "STP": str,
    "RiskoMeter": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeMaster_Join_DUPLICATED": {
    "SchemeCode": float,
    "SchemeName": str,
    "CompanyCode": float,
    "AMCCode": float,
    "SchemeType": int,
    "SchemeTypeDescription": str,
    "SchemePlanCode": float,
    "SchemePlanDescription": str,
    "SchemeClassCode": int,
    "SchemeClassDescription": str,
    "SchemeCategoryCode": int,
    "SchemeCategoryDescription": str,
    "MainCategory": str,
    "SubCategory": str,
    "LaunchDate": pd.to_datetime,
    "OpenDate": pd.to_datetime,
    "CloseDate": pd.to_datetime,
    "RedemptionDate": pd.to_datetime,
    "MinInvAmnt": float,
    "IntPrice": float,
    "IntPricUOM": int,
    "IntPUOMDes": str,
    "Size": float,
    "SizeUOM": float,
    "SizeUOMDescription": str,
    "OfferPrice": float,
    "AmountRaised": float,
    "AmountRaisedUOM": float,
    "AmountRaisedUOMDescription": str,
    "FundManagerPrefix": str,
    "FundManagerName": str,
    "ListingInformation": bool,
    "EntryLoad": str,
    "ExitLoad": str,
    "RedemptionFerq": str,
    "ISIN_Div_Payout_ISIN_Growth": str,
    "ISIN_Div_Reinvestment": str,
    "RegistrarName": str,
    "EntryLoad_WithSlab_Desc": str,
    "ExitLoad_WithSlab_Desc": str,
    "SIP_Min_Invt": str,
    "SIPEntryLoad": str,
    "SIPExitLoad": str,
    "ExpensesRatio": str,
    "PortFolioTurnOverRatio": str,
    "SWP": str,
    "STP": str,
    "RiskoMeter": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeMaster_OLD": {
    "SchemeCode": float,
    "SchemeName": str,
    "CompanyCode": float,
    "AMCCode": float,
    "SchemeType": int,
    "SchemeTypeDescription": str,
    "SchemePlanCode": float,
    "SchemePlanDescription": str,
    "SchemeClassCode": int,
    "SchemeClassDescription": str,
    "SchemeCategoryCode": int,
    "SchemeCategoryDescription": str,
    "MainCategory": str,
    "SubCategory": str,
    "LaunchDate": pd.to_datetime,
    "OpenDate": pd.to_datetime,
    "CloseDate": pd.to_datetime,
    "RedemptionDate": pd.to_datetime,
    "MinInvAmnt": float,
    "IntPrice": float,
    "IntPricUOM": int,
    "IntPUOMDes": str,
    "Size": float,
    "SizeUOM": float,
    "SizeUOMDescription": str,
    "OfferPrice": float,
    "AmountRaised": float,
    "AmountRaisedUOM": float,
    "AmountRaisedUOMDescription": str,
    "FundManagerPrefix": str,
    "FundManagerName": str,
    "ListingInformation": bool,
    "EntryLoad": str,
    "ExitLoad": str,
    "RedemptionFerq": str,
    "ISIN_Div_Payout_ISIN_Growth": str,
    "ISIN_Div_Reinvestment": str,
    "RegistrarName": str,
    "EntryLoad_WithSlab_Desc": str,
    "ExitLoad_WithSlab_Desc": str,
    "SIP_Min_Invt": str,
    "SIPEntryLoad": str,
    "SIPExitLoad": str,
    "ExpensesRatio": str,
    "PortFolioTurnOverRatio": str,
    "SWP": str,
    "STP": str,
    "RiskoMeter": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeNAVCurrentPrices": {
    "SecurityCode": float,
    "Ticker": str,
    "DateTime": pd.to_datetime,
    "NAVAmount": float,
    "RepurchaseLoad": float,
    "RepurchasePrice": float,
    "SaleLoad": float,
    "SalePrice": float,
    "PrevNAVAmount": float,
    "PrevRepurchasePrice": float,
    "PrevSalePrice": float,
    "PercentageChange": float,
    "Prev1WeekAmount": float,
    "Prev1WeekPer": float,
    "Prev1MonthAmount": float,
    "Prev1MonthPer": float,
    "Prev3MonthsAmount": float,
    "Prev3MonthsPer": float,
    "Prev6MonthsAmount": float,
    "Prev6MonthsPer": float,
    "Prev9MonthsAmount": float,
    "Prev9MonthsPer": float,
    "PrevYearAmount": float,
    "PrevYearPer": float,
    "Prev2YearAmount": float,
    "Prev2YearPer": float,
    "Prev2YearCompPer": float,
    "Prev3YearAmount": float,
    "Prev3YearPer": float,
    "Prev3YearCompPer": float,
    "Prev5YearAmount": float,
    "Prev5YearPer": float,
    "Prev5YearCompPer": float,
    "ListDate": pd.to_datetime,
    "ListAmount": float,
    "ListPer": float,
    "ListCompPer": float,
    "FiftyTwoWeekHighNAV": float,
    "FiftyTwoWeekLowNAV": float,
    "FiftyTwoWeekHighNAVDate": pd.to_datetime,
    "FiftyTwoWeekLowNAVDate": pd.to_datetime,
    "Rank": int,
},
"SchemeNAVDetails": {
    "SecurityCode": str,
    "NAVDate": pd.to_datetime,
    "NAVAmount": float,
    "RepurchaseLoad": float,
    "RepurchasePrice": float,
    "SaleLoad": float,
    "SalePrice": float,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeNAVMaster": {
    "SchemeCode": float,
    "SecurityCode": float,
    "SchemePlanCode": int,
    "SchemePlanDescription": str,
    "MappingCode": str,
    "MappingName": str,
    "IssuePrice": float,
    "Description": str,
    "IssueDate": pd.to_datetime,
    "ExpiryDate": pd.to_datetime,
    "FaceValue": float,
    "MarketLot": float,
    "ISINCode": str,
    "BenchMarkIndex": float,
    "BenchMarkIndexName": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeObjectives": {
    "SchemeCode": float,
    "SchemePlanCode": int,
    "Objectives": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemePortfolioHeader": {
    "SchemeCode": float,
    "SchemePlanCode": int,
    "HoldingDate": pd.to_datetime,
    "TotalMarketValue": float,
    "ModifiedDate": pd.to_datetime,
},
"SchemeRegistrars": {
    "CompanyCode": float,
    "AgencyName": str,
    "Address": str,
    "Address2": str,
    "Address3": str,
    "Address4": str,
    "CityName": str,
    "StateName": str,
    "PinCode": str,
    "TelephoneNumbers": str,
    "Phone2": str,
    "Phone3": str,
    "Phone4": str,
    "Fax": str,
    "Telex": str,
    "Grams": str,
    "EmailId": str,
    "Website": str,
    "ModifiedDate": pd.to_datetime,
},
"SchemeSplitsDetails": {
    "SchemeCode": float,
    "SchemePlanCode": int,
    "SplitsDate": pd.to_datetime,
    "Old_FaceValue": float,
    "New_FaceValue": float,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"SchemeTaxBenefit": {
    "SchemeCode": float,
    "SchemePlanCode": float,
    "SchemePlanDescription": str,
    "ITActSection": str,
    "TaxBenefits": str,
    "ModifyDate": pd.to_datetime,
    "DeleteFlag": bool,
    "RowId": int,
},
"SchemewisePortfolio": {
    "SchemeCode": float,
    "SchemePlanCode": int,
    "HoldingDate": pd.to_datetime,
    "InvestedCompanyCode": float,
    "InvestedCompanyName": str,
    "IndustryCode": int,
    "IndustryName": str,
    "InstrumentName": str,
    "ListingInformation": bool,
    "Quantity": float,
    "Percentage": float,
    "IsItNPA": bool,
    "MarketValue": float,
    "PortfolioUOMDescription": str,
    "Rating": str,
    "ISINCode": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
    "RowId": int,
},
"SectorDivisor": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"SectorDivisor_DUPLICATED": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"SectorIndexList": {
    "GenDate": pd.to_datetime,
    "SectorIndexName": str,
    "Sector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"SectorIndexList_DUPLICATED": {
    "GenDate": pd.to_datetime,
    "SectorIndexName": str,
    "Sector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"ShareHolding": {
    "CompanyCode": float,
    "SHPDate": pd.to_datetime,
    "Capital": float,
    "FaceValue": float,
    "NoOfShares": float,
    "Promoters": float,
    "Directors": float,
    "SubsidiaryCompanies": float,
    "OtherCompanies": float,
    "ICICI": float,
    "UTI": float,
    "IDBI": float,
    "GenInsuranceComp": float,
    "LifeInsuranceComp": float,
    "StateFinCorps": float,
    "InduFinCorpIndia": float,
    "ForeignNRI": float,
    "ForeignCollaborators": float,
    "ForeignOCB": float,
    "ForeignOthers": float,
    "ForeignInstitutions": float,
    "ForeignIndustries": float,
    "StateGovt": float,
    "CentralGovt": float,
    "GovtCompanies": float,
    "GovtOthers": float,
    "Others": float,
    "NBanksMutualFunds": float,
    "HoldingCompanies": float,
    "GeneralPublic": float,
    "Employees": float,
    "FinancialInstitutions": float,
    "ForeignPromoter": float,
    "GDR": float,
    "PersonActingInConcert": float,
    "Total": float,
    "ModifiedDate": pd.to_datetime,
},
"Splits": {
    "CompanyCode": float,
    "DateOfAnnouncement": pd.to_datetime,
    "OldFaceValue": float,
    "NewFaceValue": float,
    "RecordDate": pd.to_datetime,
    "BookClosureStartDate": pd.to_datetime,
    "BookClosureEndDate": pd.to_datetime,
    "XSDate": pd.to_datetime,
    "Remarks": str,
    "DeleteFlag": bool,
    "ModifiedDate": pd.to_datetime,
},
"SubSectorDivisor": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"SubSectorDivisor_DUPLICATED": {
    "IndexName": str,
    "IndexValue": float,
    "Divisor": float,
    "Date": pd.to_datetime,
    "OS": float,
},
"SubSectorIndexList": {
    "GenDate": pd.to_datetime,
    "SubSectorIndexName": str,
    "SubSector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"SubSectorIndexList_DUPLICATED": {
    "GenDate": pd.to_datetime,
    "SubSectorIndexName": str,
    "SubSector": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Earnings Growth": float,
},
"Subsidiaries": {
    "ParentCompanyCode": float,
    "ParentCompanyName": str,
    "CompanyCode": float,
    "CompanyName": str,
    "HoldingPercentage": float,
    "Remarks": str,
    "ModifiedDate": pd.to_datetime,
    "DeleteFlag": bool,
},
"TTM": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"TTM_BACKUP": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"TTM_DUPLICATED": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"TTM_ONWAIT": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"TTM_TEMP": {
    "CompanyCode": float,
    "YearEnding": pd.to_datetime,
    "Months": int,
    "Quarter": int,
    "Sales": float,
    "Expenses": float,
    "EBIDTA": float,
    "Interest": float,
    "Depreciation": float,
    "Extraordinary": float,
    "OPM": float,
    "Tax": float,
    "PAT": float,
    "Equity": float,
    "Reserves": float,
    "EPS": float,
    "NPM": float,
},
"index_btt_mapping": {
    "IndexName": str,
    "BTTCode": str,
},
"mf_category_mapping": {
    "scheme_code": float,
    "scheme_name": str,
    "scheme_category": str,
    "date": pd.to_datetime,
    "btt_scheme_code": str,
    "btt_scheme_category": str,
},
"mf_category_mapping_DUPLICATED": {
    "scheme_code": float,
    "scheme_name": str,
    "scheme_category": str,
    "date": pd.to_datetime,
    "btt_scheme_code": str,
    "btt_scheme_category": str,
    "Updated": str,
},
"mf_category_mapping_copy": {
    "scheme_code": float,
    "scheme_name": str,
    "scheme_category": str,
    "date": pd.to_datetime,
    "btt_scheme_code": str,
    "btt_scheme_category": str,
},
"mf_ohlc": {
    "date": pd.to_datetime,
    "btt_scheme_code": str,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": float,
},
"mf_ohlc_DUPLICATED": {
    "date": pd.to_datetime,
    "btt_scheme_code": str,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": float,
},
"nse_index_change": {
    "symbol": str,
    "change": float,
    "date": pd.to_datetime,
},
"nse_index_change_DUPLICATED": {
    "symbol": str,
    "change": float,
    "date": pd.to_datetime,
},
"ohlc_monthly": {
    "company_code": float,
    "company_name": str,
    "nse_code": str,
    "bse_code": int,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": float,
    "date": pd.to_datetime,
},
"ohlc_weekly": {
    "company_code": float,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": int,
    "date": pd.to_datetime,
},
"ohlc_weekly_DUPLICATED": {
    "company_code": float,
    "open": float,
    "high": float,
    "low": float,
    "close": float,
    "volume": int,
    "date": pd.to_datetime,
},
"temp_scheme_nav_details": {
    "SecurityCode": float,
    "NAVDate": str,
    "NAVAmount": int,
    "RepurchaseLoad": int,
    "RepurchasePrice": int,
    "SaleLoad": int,
    "SalePrice": int,
    "ModifiedDate": int,
    "DeleteFlag": int,
},
"temp_schemewiseportfolio": {
    "SchemeCode": str,
    "SchemePlanCode": str,
    "HoldingDate": str,
    "InvestedCompanyCode": int,
    "InvestedCompanyName": int,
    "IndustryCode": int,
    "IndustryName": int,
    "InstrumentName": int,
    "ListingInformation": int,
    "Quantity": int,
    "Percentage": int,
    "IsItNPA": int,
    "MarketValue": int,
    "PortfolioUOMDescription": int,
    "Rating": int,
    "ISINCode": int,
    "DeleteFlag": int,
    "ModifiedDate": int,
    "RowId": int,
},
"users": {
    "username": str,
    "password": str,
},
"FRS-NAVRank": {
    "SchemeCode": float,
    "SchemeName": str,
    "Date": pd.to_datetime,
    "Current": float,
    "1 Day": float,
    "1 Day Rank": float,
    "1 Week": float,
    "1 Week Rank": float,
    "1 Month": float,
    "1 Month Rank": float,
    "3 Month": float,
    "3 Month Rank": float,
    "6 Month": float,
    "6 Month Rank": float,
    "9 Month": float,
    "9 Month Rank": float,
    "1 Year": float,
    "1 Year Rank": float,
    "2 Year": float,
    "2 Year Rank": float,
    "3 Year": float,
    "3 Year Rank": float,
    "5 Year": float,
    "5 Year Rank": float,
    "AUM": float,
    "Scheme Rank": float,
    "btt_scheme_category": str,
    "btt_scheme_code": str,
},
"NewHighNewLow": {
    "Date": pd.to_datetime,
    "30DNH": int,
    "30DNL": int,
    "30DNHNL": int,
    "90DNH": int,
    "90DNL": int,
    "90DNHNL": int,
    "52WNH": int,
    "52WNL": int,
    "52WNHNL": int,
},
"PRS": {
    "CompanyName": str,
    "NSECode": str,
    "BSECode": int,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "Volume": int,
    "Value": float,
    "52 High": float,
    "52 Low": float,
    "52 High Date": pd.to_datetime,
    "52 Low Date": pd.to_datetime,
    "New High 52W": bool,
    "New High 90D": bool,
    "New High 30D": bool,
    "New Low 52W": bool,
    "New Low 90D": bool,
    "New Low 30D": bool,
    "RR 1D": float,
    "RR 5D": float,
    "RR 10D": float,
    "RR 30D": float,
    "RR 60D": float,
    "Change 52W": float,
    "Change 90D": float,
    "Change 30D": float,
    "RR 52W": float,
    "RR 90D": float,
    "RS 52W": float,
    "RS 90D": float,
    "RS 30D": float,
    "Combined Strength": float,
    "ISIN": str,
    "Date": pd.to_datetime,
    "RR30_Replaced": bool,
    "RR60_Replaced": bool,
    "RR90_Replaced": bool,
    "RR52W_Replaced": bool,
    "Off-High": float,
    "Off-Low": float,
    "CompanyCode": float,
    "Value Average": float,
    "Market Cap Value": float,
    "Market Cap Class": str,
    "Market Cap Rank": int,
    "PE": float,
    "PE High": float,
    "PE High Date": pd.to_datetime,
    "PE Low": float,
    "PE Low Date": pd.to_datetime,
},
"SMR": {
    "CompanyCode": float,
    "NSESymbol": str,
    "BSESymbol": int,
    "Industry": str,
    "ROEYearEnding": pd.to_datetime,
    "Months": int,
    "TTMYearEnding": pd.to_datetime,
    "SalesGrowth": float,
    "Sales Growth Rank": float,
    "NPM": float,
    "NPM Rank": float,
    "ROE": float,
    "ROE Rank": float,
    "SMR Grade": str,
    "SMR": float,
    "SMR Rank": float,
    "SMRDate": pd.to_datetime,
    "CompanyName": str,
    "ISIN": str,
},
"Consolidated_EPS": {
    "CompanyCode": float,
    "NSECode": str,
    "BSECode": int,
    "CompanyName": str,
    "ISIN": str,
    "Months": int,
    "Quarter": int,
    "YearEnding": pd.to_datetime,
    "Q1 EPS": float,
    "Q1 EPS Growth": float,
    "Q1 Sales": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
    "TTM1 EPS Growth": float,
    "TTM1 Sales Growth": float,
    "TTM2 EPS Growth": float,
    "TTM2 Sales Growth": float,
    "TTM3 EPS Growth": float,
    "TTM3 Sales Growth": float,
    "NPM": float,
    "EPS Rating": float,
    "Ranking": float,
    "EPSDate": pd.to_datetime,
    "E_ERS": float,
},
"EPS": {
    "CompanyCode": float,
    "NSECode": str,
    "BSECode": int,
    "CompanyName": str,
    "ISIN": str,
    "Months": int,
    "Quarter": int,
    "YearEnding": pd.to_datetime,
    "Q1 EPS": float,
    "Q1 EPS Growth": float,
    "Q1 Sales": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
    "TTM1 EPS Growth": float,
    "TTM1 Sales Growth": float,
    "TTM2 EPS Growth": float,
    "TTM2 Sales Growth": float,
    "TTM3 EPS Growth": float,
    "TTM3 Sales Growth": float,
    "NPM": float,
    "EPS Rating": float,
    "Ranking": float,
    "EPSDate": pd.to_datetime,
    "E_ERS": float,
},
"STANDALONE_EPS": {
    "CompanyCode": float,
    "NSECode": str,
    "BSECode": int,
    "CompanyName": str,
    "ISIN": str,
    "Months": int,
    "Quarter": int,
    "YearEnding": pd.to_datetime,
    "Q1 EPS": float,
    "Q1 EPS Growth": float,
    "Q1 Sales": float,
    "Q1 Sales Growth": float,
    "Q2 EPS": float,
    "Q2 EPS Growth": float,
    "Q2 Sales": float,
    "Q2 Sales Growth": float,
    "TTM1 EPS Growth": float,
    "TTM1 Sales Growth": float,
    "TTM2 EPS Growth": float,
    "TTM2 Sales Growth": float,
    "TTM3 EPS Growth": float,
    "TTM3 Sales Growth": float,
    "NPM": float,
    "EPS Rating": float,
    "Ranking": float,
    "EPSDate": pd.to_datetime,
    "E_ERS": float,
},
"IRS": {
    "GenDate": pd.to_datetime,
    "IndexName": str,
    "Index": str,
    "Open": float,
    "High": float,
    "Low": float,
    "Close": float,
    "MCap_Open_Index": float,
    "MCap_High_Index": float,
    "MCap_Low_Index": float,
    "MCap_Close_Index": float,
    "Volume": float,
    "PE": float,
    "EPS": float,
    "CompanyCount": int,
    "OS": float,
    "Rank": float,
    "Change": float,
    "Earnings Growth": float,
    "ff_open_sum": float,
    "ff_high_sum": float,
    "ff_low_sum": float,
    "ff_close_sum": float,
    "MCap_Open_sum": float,
    "MCap_High_sum": float,
    "MCap_Low_sum": float,
    "MCap_Close_sum": float,
    "PE High": float,
    "PE High Date": pd.to_datetime,
    "PE Low": float,
    "PE Low Date": pd.to_datetime,
}
}

# Create a connection to the database
conn = psycopg2.connect(
    host="localhost",
    database="BravisaDB",
    user="postgres",
    password="edsols" 
)

create_table_queries = []

# Create a cursor object
cur = conn.cursor()

# Get the list of all schemas excluding temporary and system schemas
schema_sql = """
SELECT schema_name 
FROM information_schema.schemata
WHERE schema_name NOT LIKE 'pg_temp_%'
AND schema_name NOT LIKE 'pg_toast%'
AND schema_name NOT LIKE 'information%'
AND schema_name NOT LIKE 'pg_catalog%'
"""
cur.execute(schema_sql)
schemas = cur.fetchall()

# Get the CREATE TABLE statements for all tables in each schema
for schema in schemas:
    if schema[0] not in ['mf_analysis', 'dash_process']:
        schema_name = schema[0]
        table_sql = f"""
        SELECT table_name 
        FROM information_schema.tables
        WHERE table_schema = '{schema_name}'
        """
        cur.execute(table_sql)
        tables = cur.fetchall()

        date_columns_collection = []
        for table in tables:
            table_name = table[0]
            # print(table_name)   

            # get the expected_data_types for the table
            columns = expected_data_types[table_name]

            # columns with pd.to_datetime 
            date_columns = [k for k, v in columns.items() if v == pd.to_datetime]
            
            # collect all the date_columns in a list
            date_columns_collection = date_columns_collection + date_columns
        
        #print all the unique values and their occurence in descending order from dates_columns_collection
        print(Counter(date_columns_collection).most_common())

                

[('ModifiedDate', 58), ('YearEnding', 23), ('Date', 8), ('GenDate', 8), ('date', 6), ('DateOfAnnouncement', 5), ('RecordDate', 5), ('BookClosureStartDate', 5), ('BookClosureEndDate', 5), ('HoldingDate', 3), ('IncorporationDate', 2), ('FirstPublicIssueDate', 2), ('DealDate', 2), ('TIMESTAMP', 2), ('SHPDate', 2), ('ModifyDate', 2), ('DividendDate', 2), ('DateTime', 1), ('ListDate', 1), ('FiftyTwoWeekHighNAVDate', 1), ('FiftyTwoWeekLowNAVDate', 1), ('AGMDate', 1), ('TRADING_DATE', 1), ('BTTDate', 1), ('Auditeddate', 1), ('BoardMeetDate', 1), ('XBDate', 1), ('EffectiveDate', 1), ('Yearending', 1), ('PriceDate', 1), ('TrailingYear', 1), ('XDDate', 1), ('MFSetUpDate', 1), ('AMCIncDate', 1), ('ShareHoldingDate', 1), ('MFDate', 1), ('NewsDate', 1), ('PE High Date', 1), ('PE Low Date', 1), ('ROEYearEnding', 1), ('TTMYearEnding', 1), ('XRDate', 1), ('BonusDate', 1), ('CategoryNavDate', 1), ('LaunchDate', 1), ('OpenDate', 1), ('CloseDate', 1), ('RedemptionDate', 1), ('NAVDate', 1), ('IssueDate', 